In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

initial_balance=1083356.67
current_balance=1083356.67


class Narration_Type(BaseModel):
    transaction_type: str = Field(description="The type of transaction from the list 'food', 'rent', 'family', 'shopping', 'self-care', 'transport', 'other', 'unknown', 'investement")
    mine: float = Field(description="The amount of money spent by the me")
    lent: float = Field(description="The amount of money lent by me. If no money lent, then 0")
    lent_by: str = Field(description="The person to whom the money is lent from the list 'pallavi', 'prateek', 'aws', 'arshad', 'none'")
    reasoning: str = Field(description="The reasoning behind the selecting transaction_type, the amount of money spent by me and lent")


def get_model(model: str = 'deepseek-r1:7b', provider: str = 'local'):
    if (provider == 'local'):
        from langchain_ollama import ChatOllama
        llm = ChatOllama(model=model, temperature=0)
        structured_llm = llm.with_structured_output(Narration_Type,
                                                    method="json_schema")
        return structured_llm
    elif (provider == 'aws'):
        from langchain_aws import ChatBedrockConverse
        import boto3
        access_key = os.getenv('ACCESS_KEY')
        secret_key = os.getenv('SECRET_KEY')
        bedrock_client = boto3.client('bedrock-runtime',
                                      region_name='us-east-1',
                                      aws_access_key_id=access_key,
                                      aws_secret_access_key=secret_key)
        llm = ChatBedrockConverse(client=bedrock_client,
                                  model=model,
                                  temperature=0)
        structured_llm = llm.with_structured_output(Narration_Type,
                                                    method="json_schema")
        return structured_llm

def get_embeddings(model:str='deepseek-r1:7b', provider:str='local'):
    if(provider == 'local'):
        from langchain_ollama import OllamaEmbeddings
        embeddings=OllamaEmbeddings(model=model)
        return embeddings
    elif(provider == 'aws'):
        from langchain_aws import BedrockEmbeddings
        import boto3
        access_key=os.getenv('ACCESS_KEY')
        secret_key=os.getenv('SECRET_KEY')
        bedrock_client=boto3.client('bedrock-runtime', region_name='us-east-1', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
        embeddings=BedrockEmbeddings(bedrock_client, model=model)
        return embeddings


In [ ]:
from langchain_core.prompts import PromptTemplate
def classify_narration(narration:str, categories:list, users:list, amount:float):
    llm=get_model()
    # llm=get_model(model='anthropic.claude-3-sonnet-20240229-v1:0', provider='aws')
    category_list=', '.join(categories)
    users_list=', '.join(users)
    prompt=PromptTemplate(
        template=
        """
        'System: You will help me classify my bank transaction into one of the following categories: {categories}.
        Also determine if entire money is spent by me or split between me and someone else. 
        Always remember to add a reasoning for your classification.
        If split, it has to be between following users: {users}'
        Here are some details related to various categories:
        - rent: Includes payments made for rent, which includes my house rent, online subscriptions like netflix, Amazon Prime, jiohotstar. 
        Washing machine rent, Maid rent. Also payment made to airtel, jio and excitel.com for mobile recharge and wifi recharge.
        - food: Includes payment related to food items like cake, pizza, burger. Also purchases from supermarket like Ratnadeep, Tata Star and DMart. Any amount less than 100 is safe to be considered as food if you
        are not able to determine the transaction category. This category will comprise the maximum number of transactions.
        - family: The user will specifically mention family in the transaction. It includes payments made to family members like my father, mother, brother, sister, wife, husband, son, daughter.
        - transport: It includes payments made for fuel, and online transport services like ola and uber.
        - self-care: It includes payments made for personal care like skin-care, hair-cut, gym. Also includes payments for fun activities like movies, comedy shows and concerts.
        - shopping: It includes payments made for shopping like clothes, shoes, bags, accessories, electronics, gadgets, furniture, home decor. The transactions include brands like addidas, nike, zara, h&m, apple, samsung, oneplus, mi, ikea, pepperfry.
        - investment: It includes payments made for investment like mutual funds, stocks, gold, silver, bitcoin, real estate. Also includes payments made for insurance like life insurance, health insurance, car insurance, bike insurance.
        - unknown: If you are unsure of where to put the transaction. Select this category.

        Here are some details related to the users who split money with me:
        - Only if the user says 'by 2' then only split the amount otherwise entire amount is spent by me.
        - All the split transactions will have 'by x' or 'By x' in the narration. This means the money will be equally split by x people.
        - If you are certain that money is lent by me and I have not mentioned any name in the transaction then select 'pallavi'
        - pallavi : My flatmate, will be used in most of the transactions related to rent, food, transport, self-care, shopping.
        - prateek : My other flatmate, rarely used in transactions.
        - aws: My company, which involves transactions related to reimbursements.
        - arshad: My colleague, involes transaction related to food.
        'User': '{narration}. The total amount spent is {amount}.'
        """,
        input_variables=['narration', 'categories', 'users', 'amount']
    )
    chain=prompt|llm
    try:
        response=chain.invoke({'narration':narration, 'categories':category_list, 'users':users_list, 'amount':amount})
        return response
    except Exception as e:  
        print("Error:",e)
    
    

In [3]:
import pandas as pd
def get_data(filename):
    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    filepath='./data/'+filename+'.DELIMITED'
    columns=['Date','Narration','Value_Date', 'Debit_Amount', 'Credit_Amount','Chq_Ref_Number', 'Closing_Balance']
    df=pd.read_csv(filepath, names=columns)
    df = df.drop(index=0)
    df = df.drop(columns=['Chq_Ref_Number', 'Value_Date'])

    # Reset the index if needed
    df = df.reset_index(drop=True)
    return df

In [ ]:
import pandas as pd
import time

def traverse_expense(categories:list, users:list, df:pd.DataFrame):
    global current_balance

    target_df = pd.DataFrame(columns=['date','narration','amount', 'mine', 'lent', 'lent_by', 'type', 'balance', 'closing_balance', 'credit', 'reasoning'])


    for index,row in df.iterrows():
        # print(row['Narration'], row['Debit_Amount'], row['Credit_Amount'])
        print("-"*80)
        if float(row['Credit_Amount'].strip()) > 0.0:
            # Create a new row for the target DataFrame
            amount=float(row['Credit_Amount'].strip())
            current_balance=current_balance+amount
            print("current_balance",current_balance)
            new_row = {
                'date': row['Date'],
                'narration':row['Narration'].lower().strip(),
                'amount': row['Credit_Amount'].strip(),
                'type': 'input_amount',
                'mine': 0,  # Set appropriate value if needed
                'lent': 0,  # Set appropriate value if needed
                'lent_by':'none',
                'balance': current_balance, # Set appropriate value if needed
                'closing_balance': row['Closing_Balance'].strip(),
                'credit':'Y',
                'reasoning':'none'
                
            }
            # Append the new row to the target DataFrame
            target_df = pd.concat([target_df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            amount=float(row['Debit_Amount'].strip())
            current_balance=current_balance-amount
            print("current_balance",current_balance)

            response=classify_narration(narration=row['Narration'].lower(),categories=categories, users=users, amount=row['Debit_Amount'])
            
            transaction_type=response.transaction_type.lower()
            mine=response.mine
            lent=response.lent
            lent_by=response.lent_by
            reasoning=response.reasoning
            print(row['Narration'].strip(),reasoning)



            new_row = {
                'date': row['Date'],
                'narration':row['Narration'].lower().strip(),
                'amount': row['Debit_Amount'].strip(),
                'type': transaction_type,
                'mine': mine,  # Set appropriate value if needed
                'lent': lent,  # Set appropriate value if needed
                'lent_by':lent_by,
                'balance': current_balance, # Set appropriate value if needed
                'closing_balance': row['Closing_Balance'].strip(),
                'credit':'N',
                'reasoning':reasoning
            }
            # Append the new row to the target DataFrame
            target_df = pd.concat([target_df, pd.DataFrame([new_row])], ignore_index=True)
        
        print("-"*80)

    target_df.to_csv('./data/raw/target_df.csv', index=False) 
    
    return target_df

In [5]:
def get_cleaned_data():
    cleaned_df=pd.read_csv('./data/cleaned/cleaned_df.csv')
    target_df=pd.read_csv('./data/raw/target_df.csv')
    combined_df = pd.concat([cleaned_df, target_df], ignore_index=True)
    combined_df.to_csv('./data/cleaned/cleaned_df.csv', index=False)
    return combined_df


In [6]:
def get_input_amount():
    cleaned_df=pd.read_csv('./data/cleaned/cleaned_df.csv')
    input_df=cleaned_df[cleaned_df['type']=='input_amount']
    input_df.to_csv('./data/cleaned/input_df.csv', index=False)
    return input_df


In [7]:
def curate_data()->pd.DataFrame:
    cleaned_df=pd.read_csv('./data/cleaned/cleaned_df.csv')
    input_df=pd.read_csv('./data/cleaned/input_df.csv')
    category_columns = ['my_expenses','category_food', 'category_rent', 'category_family', 'category_shopping', 
                        'category_self-care', 'category_transport', 'category_other','catagory_investment',
                        'user_pallavi', 'user_prateek', 'user_aws', 'user_arshad']
    df=pd.DataFrame(columns=category_columns)

    total_mine_spent = cleaned_df[cleaned_df['mine'] > 0]['mine'].sum()
    total_lent_spent = cleaned_df[cleaned_df['lent'] > 0]['lent'].sum()

    total_food_spent = cleaned_df[cleaned_df['type'] == 'food']['amount'].sum()
    total_rent_spent = cleaned_df[cleaned_df['type'] == 'rent']['amount'].sum()
    total_family_spent = cleaned_df[cleaned_df['type'] == 'family']['amount'].sum()
    total_shopping_spent = cleaned_df[cleaned_df['type'] == 'shopping']['amount'].sum()
    total_self_care_spent = cleaned_df[cleaned_df['type'] == 'self-care']['amount'].sum()
    total_transport_spent = cleaned_df[cleaned_df['type'] == 'transport']['amount'].sum()
    total_other_spent = cleaned_df[cleaned_df['type'] == 'other']['amount'].sum()
    total_investment_spent = cleaned_df[cleaned_df['type'] == 'investment']['amount'].sum()

    total_paid_pallavi=input_df[input_df['lent_by']=='pallavi']['amount'].sum()
    total_paid_prateek=input_df[input_df['lent_by']=='prateek']['amount'].sum() 
    total_paid_aws=input_df[input_df['lent_by']=='aws']['amount'].sum()
    total_paid_arshad=input_df[input_df['lent_by']=='arshad']['amount'].sum()

    total_owe_pallavi=cleaned_df[cleaned_df['lent_by']=='pallavi' ]['lent'].sum()
    total_owe_prateek=cleaned_df[cleaned_df['lent_by']=='prateek' ]['lent'].sum()
    total_owe_aws=cleaned_df[cleaned_df['lent_by']=='aws' ]['lent'].sum()
    total_owe_arshad=cleaned_df[cleaned_df['lent_by']=='arshad' ]['lent'].sum()

    df.loc[0, 'my_expenses'] = total_mine_spent
    df.loc[0, 'lent'] = total_lent_spent
    
    df.loc[0, 'category_food'] = total_food_spent
    df.loc[0, 'category_rent'] = total_rent_spent
    df.loc[0, 'category_family'] = total_family_spent
    df.loc[0, 'category_shopping'] = total_shopping_spent       
    df.loc[0, 'category_self-care'] = total_self_care_spent
    df.loc[0, 'category_transport'] = total_transport_spent
    df.loc[0, 'category_other'] = total_other_spent
    df.loc[0, 'catagory_investment'] = total_investment_spent
    df.loc[0, 'user_pallavi'] = total_owe_pallavi- total_paid_pallavi
    df.loc[0, 'user_prateek'] = total_owe_prateek-total_paid_prateek
    df.loc[0, 'user_aws'] = total_owe_aws-total_paid_aws
    df.loc[0, 'user_arshad'] = total_owe_arshad-total_paid_arshad

    df.to_csv('./data/curated/curated_df.csv', index=False)
    


    
    return df

In [8]:
def check_balance():
    df_curated=pd.read_csv('./data/curated/curated_df.csv')
    mine=df_curated['my_expenses'].values[0]
    lent=df_curated['user_pallavi'].values[0]+df_curated['user_prateek'].values[0]+df_curated['user_aws'].values[0]+df_curated['user_arshad'].values[0]
    global initial_balance
    global current_balance

    print("Initial Balance:", initial_balance)
    print("Current Balance:", current_balance)
    print("My Expenses:",mine)
    print("Lent:",lent)

    diff=current_balance-(initial_balance-mine-lent)

    if(diff<500.00):
        print("The balance is correct")
    else:
        print("The balance is incorrect by amount:", current_balance-(initial_balance-mine-lent))
    

In [9]:
categories=['food', 'rent', 'family', 'shopping', 'self-care', 'transport', 'other', 'unknown']
users=['pallavi', 'prateek', 'aws', 'arshad']
df=get_data('march6')
target_df=traverse_expense(categories, users, df)
get_cleaned_data()

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


/var/folders/nx/wvnbfw697mbf57dq4vcl2nbm0000gn/T/ipykernel_33856/2697297770.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_df = pd.concat([target_df, pd.DataFrame([new_row])], ignore_index=True)


UPI-WWW EXCITEL COM-EXCITEL.PAYU@HDFCBANK-HDFC0000499-100079521366-UPI TRANSACTION This transaction is for paying the Excitel broadband internet bill, which falls under the rent category as per the given details. The narration mentions "excitel.com" which is the internet service provider. Since there is no mention of splitting the amount, I have assumed the entire amount of 824.82 is spent by me.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
UPI-JIOHOTSTAR-HOTSTARONLINE@YBL-YESB0YBLUPI-506055997561-COLLECT REQUEST FR This transaction is for paying the JioHotstar online subscription, which falls under the rent category as per the given details. The entire amount of 79 is spent by me as there is no mention of splitting the cost with anyone else.
--------------------------------------------------------------------------------
-------------------------------------------------

/var/folders/nx/wvnbfw697mbf57dq4vcl2nbm0000gn/T/ipykernel_33856/1077704377.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([cleaned_df, target_df], ignore_index=True)


,amount,mine,lent,balance,date,narration,type,lent_by,closing_balance,credit,reasoning
0,35000.00,0.00,0.0,1118356.67,28/02/25,upi-prateek ranjan dutta-dada.prateek.00358@okhdfcbank-hdfc0004809-100743026595-rent maid plus 15k,input_amount,none,1118856.67,Y,none
1,824.82,824.82,0.0,1117531.85,01/03/25,upi-www excitel com-excitel.payu@hdfcbank-hdfc0000499-100079521366-upi transaction,rent,none,1118031.85,N,"This transaction is for paying the Excitel broadband internet bill, which falls under the rent category as per the given details. The narration mentions ""excitel.com"" which is the internet service provider. Since there is no mention of splitting the amount, I have assumed the entire amount of 824.82 is spent by me."
2,79.00,79.00,0.0,1117452.85,01/03/25,upi-jiohotstar-hotstaronline@ybl-yesb0yblupi-506055997561-collect request fr,rent,none,1117952.85,N,"This transaction is for paying the JioHotstar online subscription, which falls under the rent category as per the given details. The entire amount of 79 is spent by me as there is no mention of splitting the cost with anyone else."
3,500.00,500.00,0.0,1116952.85,01/03/25,upi-sandhya fuel station-paytm-65718951@ptys-yesb0ptmupi-100786433017-upi,transport,none,1117452.85,N,"The narration mentions ""fuel station"", which indicates that the transaction is related to transport and specifically for fuel. Since there is no mention of splitting the amount, the entire 500 is spent by me."
4,10.00,10.00,0.0,1116942.85,01/03/25,upi-adigoppula shravan k-paytmqr5yzxcj@ptys-yesb0ptmupi-100787095809-upi,food,none,1117442.85,N,"Based on the narration, this transaction appears to be a UPI payment of Rs. 10. Since the amount is less than Rs. 100 and there is no specific mention of the purpose, it is reasonable to classify this as a food-related transaction."
5,3000.00,3000.00,0.0,1113942.85,01/03/25,nwd-435584xxxxxx5455-id043212-hyderabad,unknown,none,1114442.85,N,"Based on the narration ""nwd-435584xxxxxx5455-id043212-hyderabad"", it is not clear what this transaction is for. There are no keywords or identifiers that match any of the provided categories like food, rent, family, shopping, self-care, transport, investment or other. Therefore, I have classified this transaction as unknown. The entire amount of 3000 is spent by me as there is no mention of splitting the cost."
6,90454.00,45227.00,45227.0,1023488.85,01/03/25,upi-md arshad ahmed-itzzarshad19@okaxis-utib0000376-100795816337-half amount,food,arshad,1023988.85,N,"The narration mentions ""arshad ahmed"" and ""half amount"", indicating that the transaction is related to food and the amount is split equally between me and Arshad, who is mentioned as a colleague for food-related transactions."
7,1000.00,500.00,500.0,1022488.85,01/03/25,upi-jamalpur vijay-q786072912@ybl-yesb0yblupi-100832556878-choclate room by 2,food,pallavi,1022988.85,N,"The narration mentions ""choclate room"" which seems to be a food establishment. The amount is split ""by 2"" indicating it is shared between me and someone else, most likely my flatmate pallavi for a food expense."


In [10]:
get_input_amount()

,amount,mine,lent,balance,date,narration,type,lent_by,closing_balance,credit,reasoning
0,35000.0,0.0,0.0,1118356.67,28/02/25,upi-prateek ranjan dutta-dada.prateek.00358@okhdfcbank-hdfc0004809-100743026595-rent maid plus 15k,input_amount,none,1118856.67,Y,none


In [317]:
# before running this cell clean input df and cleaned df
cleaned_df=pd.read_csv('./data/cleaned/cleaned_df.csv')
cleaned_df.head(10)


,amount,mine,lent,balance,date,narration,type,lent_by,closing_balance,credit,reasoning
0,35000.00,0.0,0.00,997121.03,28/02/25,upi-prateek ranjan dutta-dada.prateek.00358@okhdfcbank-hdfc0004809-100743026595-rent maid plus 15k,input_amount,none,1118856.67,Y,none
1,824.82,0.0,824.82,996296.21,01/03/25,upi-www excitel com-excitel.payu@hdfcbank-hdfc0000499-100079521366-upi transaction,rent,aws,1118031.85,N,"This transaction is for paying the Excitel broadband internet bill, which falls under the rent category as per the given details. The narration mentions ""excitel.com"" which is the internet service provider. Since there is no mention of splitting the amount, I have assumed the entire amount of 824.82 is spent by me."
2,79.00,79.0,0.00,996217.21,01/03/25,upi-jiohotstar-hotstaronline@ybl-yesb0yblupi-506055997561-collect request fr,rent,none,1117952.85,N,"The narration mentions ""jiohotstar"" which is an online subscription service, so this transaction falls under the rent category as per the given details. The entire amount of 79.0 is spent by me as there is no mention of splitting the cost."
3,500.00,500.0,0.00,995717.21,01/03/25,upi-sandhya fuel station-paytm-65718951@ptys-yesb0ptmupi-100786433017-upi,transport,none,1117452.85,N,"The narration mentions ""fuel station"", which indicates that the transaction is related to transport and specifically for fuel. Since there is no mention of splitting the amount, I assume the entire 500 was spent by me."
4,10.00,10.0,0.00,995707.21,01/03/25,upi-adigoppula shravan k-paytmqr5yzxcj@ptys-yesb0ptmupi-100787095809-upi,food,none,1117442.85,N,"The transaction narration does not mention any specific category, and the amount is less than 100. Based on the guidelines provided, transactions with an amount less than 100 can be safely categorized as food if the category is unclear. Therefore, I have classified this transaction as food, with the entire amount spent by me."
5,3000.00,3000.0,0.00,992707.21,01/03/25,nwd-435584xxxxxx5455-id043212-hyderabad,unknown,none,1114442.85,N,"Based on the narration ""nwd-435584xxxxxx5455-id043212-hyderabad"", it is not clear what this transaction is for. There are no keywords or identifiers that match any of the provided categories like food, rent, family, shopping, self-care, transport, investment or other. Therefore, I have classified this transaction as unknown. The entire amount of 3000 is spent by me as there is no mention of splitting the cost."
6,90454.00,90454.0,0.00,902253.21,01/03/25,upi-md arshad ahmed-itzzarshad19@okaxis-utib0000376-100795816337-half amount,food,none,1023988.85,N,MacBook Pro purchase
7,1000.00,500.0,500.00,901253.21,01/03/25,upi-jamalpur vijay-q786072912@ybl-yesb0yblupi-100832556878-choclate room by 2,food,pallavi,1022988.85,N,"The narration mentions ""choclate room by 2"", indicating that the transaction is related to food and the amount is split between two people. Since no specific name is mentioned for the other person, I assume it is your flatmate pallavi based on the given information."


In [318]:
curated_df=curate_data()
check_balance()

Initial Balance: 1022988.8500000001
Current Balance: 901253.2100000001
My Expenses: 94543.0
Lent: -33675.18
The balance is correct
